In [1]:
!pip install pandas
!pip install geopandas
!pip install plotly
!pip install json

   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
    --------------------------------------- 0.3/15.9 MB ? eta -:--:--
    --------------------------------------- 0.3/15.9 MB ? eta -:--:--
   - -------------------------------------- 0.8/15.9 MB 1.2 MB/s eta 0:00:13
   --- ------------------------------------ 1.6/15.9 MB 2.0 MB/s eta 0:00:07
   ------ --------------------------------- 2.6/15.9 MB 2.8 MB/s eta 0:00:05
   ---------- ----------------------------- 4.2/15.9 MB 3.6 MB/s eta 0:00:04
   -------------- ------------------------- 5.8/15.9 MB 4.4 MB/s eta 0:00:03
   ------------------- -------------------- 7.9/15.9 MB 5.2 MB/s eta 0:00:02
   ------------------------- -------------- 10.0/15.9 MB 5.9 MB/s eta 0:00:02
   ------------------------------ ------

In [14]:
pip install --upgrade nbformat


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import json


In [6]:
shapefile_path = 'C:/Users/WORK/Downloads/layerdistrito/LAYER_DISTRITO/DEINFO_DISTRITO.shp'
gdf_distritos = gpd.read_file(shapefile_path)
mapa = pd.read_csv('C:/Users/WORK/C UNINTER VSCODE/Projetos VSCODE/Previsão do tempo/teste_mapa2.csv')
gdf_distritos.rename(columns={'NOME_DIST': 'Distrito'}, inplace=True)

In [7]:
def ajustar_nome_distrito(nome):
    nome = nome.upper().replace('_', ' ')
    nome = nome.replace('M BOI MIRIM', 'JAGUARE')  # Mapear para JD ANGELA
    nome = nome.replace('VILA CURUÇA', 'VILA CURUCA')
    nome = nome.replace('JARDIM SAO LUIS', 'JD SAO LUIS')
    nome = nome.replace('JARDIM HELENA', 'JD HELENA')
    nome = nome.replace('JARDIM ANGELA', 'JD ANGELA')
    nome = nome.replace('CIDADE ', 'CID ')
    nome = nome.replace('ALTO DE PINHEIROS', 'ALTO DE PINHEIROS')
    nome = nome.replace('FREGUESIA DO O', 'FREGUESIA DO O')
    nome = nome.replace('JARDIM PAULISTA', 'JD PAULISTA')
    # Remover acentos e caracteres especiais
    import unicodedata
    nome = unicodedata.normalize('NFKD', nome).encode('ASCII', 'ignore').decode('ASCII')
    nome = nome.strip()
    return nome
mapa['Distrito'] = mapa['Distrito'].apply(ajustar_nome_distrito)


In [8]:
gdf_filtrado = gdf_distritos.merge(mapa, on='Distrito')
gdf_filtrado['DATA_CRIAC'] = gdf_filtrado['DATA_CRIAC'].astype(str)
gdf_filtrado['Data_Hora'] = mapa['Data_Hora']
# Converter o GeoDataFrame para GeoJSON
gdf_filtrado = gdf_filtrado.set_geometry('geometry')
gdf_filtrado = gdf_filtrado.to_crs(epsg=4326)  # Garantir que está no sistema de coordenadas WGS84
gdf_filtrado['Probabilidade_Queda'] = gdf_filtrado['Probabilidade_Classe_1'] + gdf_filtrado['Probabilidade_Classe_2']

geojson_distritos = json.loads(gdf_filtrado.to_json())


In [14]:

# Extrair os atributos de previsão em um DataFrame
dados_previsao = []

for feature in geojson_distritos['features']:
    properties = feature['properties']
    dados_previsao.append({
        'Distrito': properties['Distrito'],
        'Data_Hora': properties['Data_Hora'],
        'Probabilidade_Classe_0': properties['Probabilidade_Classe_0'],
        'Probabilidade_Classe_1': properties['Probabilidade_Classe_1'],
        'Probabilidade_Classe_2': properties['Probabilidade_Classe_2'],
        'Probabilidade_Queda': properties['Probabilidade_Queda']
    })

# Converter em DataFrame para salvar como CSV (dados dinâmicos)
df_previsao = pd.DataFrame(dados_previsao)
df_previsao.to_csv("dados_previsao.csv", index=False)

# Remover os dados dinâmicos do GeoJSON para obter apenas o layout (geometria fixa)
for feature in geojson_distritos['features']:
    feature['properties'] = {
        'Distrito': feature['properties']['Distrito'],
        'SIGLA_DIST': feature['properties']['SIGLA_DIST'],
        'COD_DIST': feature['properties']['COD_DIST'],
        'COD_SUB': feature['properties']['COD_SUB']
    }

# Salvar o GeoJSON com o layout fixo
with open('layout_geojson_simplificado.geojson', 'w') as f:
    json.dump(geojson_distritos, f)


In [8]:
# Gerar o mapa com animação temporal
gdf_filtrado['Probabilidade_Queda_Pct'] = (gdf_filtrado['Probabilidade_Queda'] * 100).round(2)

fig = px.choropleth_mapbox(
    gdf_filtrado,
    geojson=geojson_distritos,
    locations='Distrito',
    featureidkey='properties.Distrito',
    color='Probabilidade_Queda',
    color_continuous_scale='YlOrRd',
    range_color=(0, 1),
    mapbox_style='carto-positron',
    zoom=10,
    center={'lat': -23.55052, 'lon': -46.63331},
    opacity=0.7,
    labels={'Probabilidade_Queda': 'Probabilidade de Queda (%)'},
    hover_data={
        'Distrito': True,
        'Probabilidade_Queda_Pct': True
    },
    animation_frame='Data_Hora',
    animation_group='Distrito'
)

# Atualizar o layout
fig.update_layout(
    title_text='Probabilidade de Queda de Energia nos Distritos de São Paulo ao Longo do Tempo',
    title_x=0.5,
    margin={'r':0, 't':50, 'l':0, 'b':0},
    coloraxis_colorbar={
        'title': 'Probabilidade (%)',
        'tickvals': [0, 0.25, 0.5, 0.75, 1],
        'ticktext': ['0%', '25%', '50%', '75%', '100%']
    }
)

# Ajustar a velocidade da animação
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 50 
fig.show()


In [ ]:
gdf_filtrado['Probabilidade_Queda_Pct'] = (gdf_filtrado['Probabilidade_Queda'] * 100).round(2).astype(str) + '%'

gdf_filtrado['Prob_Class_0_Pct'] = (gdf_filtrado['Probabilidade_Classe_0'] * 100).round(2).astype(str) + '%'
gdf_filtrado['Prob_Class_1_Pct'] = (gdf_filtrado['Probabilidade_Classe_1'] * 100).round(2).astype(str) + '%'
gdf_filtrado['Prob_Class_2_Pct'] = (gdf_filtrado['Probabilidade_Classe_2'] * 100).round(2).astype(str) + '%'

gdf_filtrado['Probabilidade_Queda_Fill'] = gdf_filtrado['Probabilidade_Queda'].fillna(-0.1)

custom_data = gdf_filtrado[['Prob_Class_0_Pct','Prob_Class_1_Pct','Prob_Class_2_Pct']]

fig = px.choropleth_mapbox(
    gdf_filtrado,
    geojson=geojson_distritos,
    locations='Distrito',
    featureidkey='properties.Distrito',
    color='Probabilidade_Queda',
    color_continuous_scale='YlOrRd',
    range_color=(0, 1),
    mapbox_style='carto-positron',
    zoom=10,
    center={'lat': -23.55052, 'lon': -46.63331},
    opacity=0.7,
    labels={'Probabilidade_Queda_Fill': 'Probabilidade de Queda (%)'},
    hover_data=None,
    animation_frame='Data_Hora',
    animation_group='Distrito'
)

fig.update_layout(
    title={
        'text': 'Probabilidade de Queda de Energia nos Distritos de São Paulo nas Próximas Horas',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size':24, 'family': 'Arial'}
    },
    margin={'r':0, 't':50, 'l':0, 'b':0},
    coloraxis_colorbar={
        'title': 'Probabilidade',
        'tickvals': [-0.1,0,0.25,0.5,0.75,1.0],
        'ticktext': ['Sem Dados','0%','25%','50%','75%','100%'],
        'titlefont': {'size': 16},
        'tickfont': {'size': 12}
    }
)

hovertemplate = {
    '<b>%{location}</b><br>'
    'Prob. Sem Interrupção: %{customdata[0]}<br>'
    'Prob. Interrupção em 1H: %{customdata[1]}<br>'
    'Prob. Interrupcão Ocorrendo: %{customdata[2]}<br>'
    'Data/Hora: %{animation_frame}<extra></extra>'
}
fig.show()